# Statistical Analysis of the Lab Normality Paper

### Author: Song Xu

## Setting up variables

- Assign Data Set and Type here

In [1]:
curr_version = '10000-episodes-lastnormal'
inverse01 = True # Setting 'True' to interpret 'Normal' as 'Negative'

inverse_maker = '_inversed01' if inverse01 else ''

## Permutation Test

In [3]:
import LocalEnv

from scripts.LabTestAnalysis.machine_learning.LabNormalityPredictionPipeline \
        import NON_PANEL_TESTS_WITH_GT_500_ORDERS
from medinfo.ml.SupervisedClassifier import SupervisedClassifier


In [4]:
labs=NON_PANEL_TESTS_WITH_GT_500_ORDERS
algs=SupervisedClassifier.SUPPORTED_ALGORITHMS

In [1]:
from medinfo.db import DBUtil
from medinfo.db.Model import SQLQuery

In [223]:
query = SQLQuery()
query.addSelect('COUNT(DISTINCT po.pat_id)')
query.addFrom('stride_patient AS p')
query.addFrom('stride_order_proc AS po')
query.addWhere('po.pat_id = p.pat_id')
query.addWhere("po.order_time >= '2008-01-01'")
query.addWhere("po.order_time < '2017-07-01'")
results_patCnt = DBUtil.execute(query)

In [224]:
results_patCnt

[[116637L]]

In [3]:
# Number of inpatients investigated in the intensity study
query = SQLQuery()
query.addSelect('gender, COUNT(DISTINCT po.pat_id)')
query.addFrom('stride_patient AS p')
query.addFrom('stride_order_proc AS po')
query.addWhere('po.pat_id = p.pat_id')
query.addWhere("po.order_time >= '2014-07-01'")
query.addWhere("po.order_time < '2017-07-01'")
query.addGroupBy('gender')
results_genderCnt = DBUtil.execute(query)

[2019-07-21 12:44:29,741 INFO] DBUtil.connection.63: Preparing DB Connection to tmp@localhost as postgres


In [4]:
results_genderCnt

[['FEMALE', 4055L],
 ['Female', 18609L],
 ['MALE', 3642L],
 ['Male', 18374L],
 ['Unknown', 1L],
 [None, 29L]]

In [2]:
# Number of inpatients investigated in the intensity study
query = SQLQuery()
query.addSelect('EXTRACT(YEAR FROM order_time) - p.birth_year AS age, COUNT(DISTINCT p.pat_id), gender')
query.addFrom('stride_patient AS p')
query.addFrom('stride_order_proc AS po')
query.addWhere('po.pat_id = p.pat_id')
query.addWhere("po.order_time >= '2008-01-01'")
query.addWhere("po.order_time < '2017-07-01'")
query.addGroupBy('age')
query.addGroupBy('gender')
results_ageSexCnt = DBUtil.execute(query)

[2019-08-06 20:56:45,886 INFO] DBUtil.connection.63: Preparing DB Connection to tmp@localhost as postgres


In [3]:
import pandas as pd
df_ageSexCnt = pd.DataFrame(results_ageSexCnt)
df_ageSexCnt

,0,1,2
0,0.0,1,MALE
1,1.0,1,MALE
2,3.0,1,Female
3,10.0,1,MALE
4,10.0,1,Male
5,12.0,1,Female
6,12.0,2,MALE
7,13.0,1,MALE
8,13.0,1,Male
9,14.0,9,FEMALE


In [4]:
df_ageMaleCnt = df_ageSexCnt[df_ageSexCnt[2].isin(['MALE', 'Male'])]
df_ageFemaleCnt = df_ageSexCnt[df_ageSexCnt[2].isin(['FEMALE', 'Female'])]

In [5]:
(df_ageMaleCnt[0]*df_ageMaleCnt[1]).sum() / df_ageMaleCnt[1].sum() #.mean()

58.95033171223453

In [6]:
(df_ageFemaleCnt[0]*df_ageFemaleCnt[1]).sum() / df_ageFemaleCnt[1].sum() #.mean()

58.7611788750179

In [9]:
import numpy as np

all_cnts = df_ageMaleCnt[1].values
all_ages = df_ageMaleCnt[0].values

all_males = []
for i in range(len(all_ages)):
    for j in range(all_cnts[i]):
        all_males.append(all_ages[i])
print np.std(all_males)

18.104354834545635


In [10]:
all_cnts = df_ageFemaleCnt[1].values
all_ages = df_ageFemaleCnt[0].values

all_females = []
for i in range(len(all_ages)):
    for j in range(all_cnts[i]):
        all_females.append(all_ages[i])
print np.std(all_females)

18.967577523036514


In [220]:
df_order_1day = pd.read_csv("/Users/songxu/healthrex/CDSS/scripts/LabTestAnalysis/lab_statistics/Fig2_Order_Intensities/Order_Intensities_panel_labs_order_1day.csv")

In [222]:
df_order_1day['< 24 hrs'].sum()

792397